In [119]:
import pandas as pd
import pyspark
from pyspark.sql import SparkSession, functions as F
import re
import numpy as np

In [120]:
startpath ="_Star_Ratings_and_Display_Measures/"
fallpath ="_Star_Ratings_Fall_Release/"
springpath ="_Star_Ratings_Spring_Release/"
cpath="_Part_C"
dpath="_Part_D"
finalpath ="_Report_Card_Master_Table.xlsx"
year = "2014"



In [121]:
firstfive = ["Contract Number","Organization Type","Contract Name","Organization Marketing Name","Parent Organization"]
lasttwo =["Year","Overall"]

In [122]:
y = 2017
path = f"./Data/{y}{startpath}{y}{fallpath}{y}{finalpath}"
dfms = pd.read_excel(path,"Measure_Stars",header=2)
dfms = dfms.iloc[1:]
for x in range(5,len(dfms.keys())):
    dfms =dfms.rename(columns= {dfms.keys()[x] : re.split(r'\d+:',dfms.keys()[x])[0]+re.split(r'\d+:',dfms.keys()[x])[1]})
for x in range(5):
    dfms = dfms.rename(columns={f"Unnamed: {x}":firstfive[x]})
dfms

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Annual Flu Vaccine,C Improving or Maintaining Physical Health,C Improving or Maintaining Mental Health,...,D Beneficiary Access and Performance Problems,D Drug Plan Quality Improvement,D Rating of Drug Plan,D Getting Needed Prescription Drugs,D MPF Price Accuracy,D High Risk Medication,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR
1,E0654,Employer/Union Only Direct Contract PDP,IBT VOLUNTARY EMPLOYEE BENEFITS TRUST,TEAMStar Medicare Part D Prescription Drug Pro...,IBT Voluntary Employee Benefits Trust,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,5,4,5,Not enough data available,4,3,4,3,3
2,E2630,Employer/Union Only Direct Contract PDP,C AND O EMPLOYEES' HOSPITAL ASSOCIATION,C and O Employees Hospital Association,C & O Employees' Hospital Association,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,4,3,5,5,Not enough data available,3,3,4,4,2
3,E3014,Employer/Union Only Direct Contract PDP,PSERS HOP PROGRAM,Pennsylvania Public School Employees Retiremen...,Commonwealth of PA Pub Schools Retirement System,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,4,5,5,Not enough data available,5,4,5,4,3
4,E4744,Employer/Union Only Direct Contract PDP,MODOT/MSHP MEDICAL AND LIFE INSURANCE PLAN,MISSOURI DEPARTMENT OF TRANSPORTATION,Missouri Highways and Transportation Commission,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,4,4,5,Not enough data available,4,4,5,4,3
5,E7316,Employer/Union Only Direct Contract PDP,UNION PACIFIC RAILROAD EMPLOYES HEALTH SYSTEMS,Union Pacific Railroad Employes Health Systems,Union Pacific Railroad Employes Health Systems,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,2,5,2,Not enough data available,3,1,1,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
603,S7694,PDP,ENVISION INSURANCE COMPANY,EnvisionRx Plus,Rite Aid Corporation,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,3,4,3,3,5,5,2,1,2,3
604,S8067,PDP,AVALON INSURANCE COMPANY,Avalon Insurance Company,Capital BlueCross,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,1,4,3,3,5,5,5,5,4,4
605,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,4,2,2,Not enough data available,2,3,4,3,5
606,S9579,PDP,TRANSAMERICA LIFE INSURANCE COMPANY,Transamerica Life Insurance Company,Aegon N.V.,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure,...,5,3,5,5,5,3,3,3,3,3


In [123]:
def get_measure_stars(path):
    dfms = pd.read_excel(path,"Measure_Stars",header=2)
    dfms = dfms.iloc[1:]
    for x in range(5,len(dfms.keys())):
        dfms =dfms.rename(columns= {dfms.keys()[x] : re.split(r'\d+:',dfms.keys()[x])[0]+re.split(r'\d+:',dfms.keys()[x])[1]})
    for x in range(5):
        dfms = dfms.rename(columns={f"Unnamed: {x}":firstfive[x]})
    return dfms

In [124]:
def get_domain_stars(path):
    dfds = pd.read_excel(path,"Domain_Stars",header=1)
    return dfds

In [125]:
def get_summary_rating(path):
    dfsr = pd.read_excel(path,"Summary_Rating",header=1)
    dfsr = dfsr.drop(columns="Sanction Deduction",errors='ignore')
    return dfsr

In [126]:
def get_early(y):
    fullpath = f"./Data/{y}{startpath}{y}{fallpath}{y}{finalpath}"
    dfmsf = get_measure_stars(fullpath)
    dfdsf = get_domain_stars(fullpath)
    dfsrf = get_summary_rating(fullpath)
    dff = pd.merge(dfmsf,dfdsf,on=firstfive,how='left')
    dff = pd.merge(dff,dfsrf,on=firstfive,how='left')
    dff["Year"] = f"{y} Fall"
    dff = dff.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    fullpath = f"./Data/{y}{startpath}{y}{springpath}{y}{finalpath}"
    dfmss = get_measure_stars(fullpath)
    dfdss = get_domain_stars(fullpath)
    dfsrs = get_summary_rating(fullpath)
    dfs = pd.merge(dfmss,dfdss,on=firstfive,how='left')
    dfs = pd.merge(dfs,dfsrs,on=firstfive,how='left')
    dfs["Year"] = f"{y} Spring"
    dfs = dfs.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    df = pd.concat([dff,dfs])
    df = df.drop(columns = "2017 Disaster %",errors='ignore')
    return df

In [127]:
def get_late(y):
    fullpath = f"./Data/{y}{startpath}{y}{finalpath}"
    dfmsc = get_measure_stars(fullpath)
    dfdsc = get_domain_stars(fullpath)
    dfsrc = get_summary_rating(fullpath)
    #drop disaster columns
    dfsrc = dfsrc.drop(columns=dfsrc.columns[[6,7]],axis = 1)
    df = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    df = pd.merge(df,dfsrc,on=firstfive,how='left')
    df = df.rename(columns={f"{y} Part C Summary":"Part C Summary",f"{y} Overall":"Overall",f"{y} Part D Summary":"Part D Summary"})
    df["Year"] = f"{y}"
    return    df

In [128]:
def standardize_data():
    year = 2014
    years = [x for x in range(2015,2026)]        
    fullpath = f"./Data/{year}{startpath}{year}{fallpath}{year}{cpath}{finalpath}"
    dfmsc = get_measure_stars(fullpath)
    dfdsc = get_domain_stars(fullpath)
    dfsrc = get_summary_rating(fullpath)
    df = pd.merge(dfmsc,dfdsc,on=firstfive,how='left')
    df = pd.merge(df,dfsrc,on=firstfive,how='left')
    fullpath = f"./Data/{year}{startpath}{year}{fallpath}{year}{dpath}{finalpath}"
    dfmsd = get_measure_stars(fullpath)
    dfdsd = get_domain_stars(fullpath)
    dfsrd = get_summary_rating(fullpath)
    dfd = pd.merge(dfmsd,dfdsd,on=['Contract Number'],how='left')
    dfd = pd.merge(dfd,dfsrd,on=['Contract Number'],how='left')
    dfd = dfd.drop(columns=["Organization Type_x","Contract Name_x","Organization Marketing Name_x","Parent Organization_x","Organization Type_y","Contract Name_y","Organization Marketing Name_y","Parent Organization_y"])
    df = pd.merge(df,dfd,on=["Contract Number","Organization Type","Contract Name","Organization Marketing Name","Parent Organization","SNP"],how='left')
    df = df.rename(columns={f"{year} Part C Summary Rating":"Part C Summary",f"{year} Overall Rating":"Overall",f"{year} Part D Summary Rating":"Part D Summary"})
    fullpath = f"./Data/{year}{startpath}{year}{springpath}{year}{cpath}{finalpath}"
    dfmsc1 = get_measure_stars(fullpath)
    dfdsc1 = get_domain_stars(fullpath)
    dfsrc1 = get_summary_rating(fullpath)
    df1 = pd.merge(dfmsc1,dfdsc1,on=firstfive,how='left')
    df1 = pd.merge(df1,dfsrc1,on=firstfive,how='left')
    fullpath = f"./Data/{year}{startpath}{year}{springpath}{year}{dpath}{finalpath}"
    dfmsd1 = get_measure_stars(fullpath)
    dfdsd1 = get_domain_stars(fullpath)
    dfsrd1 = get_summary_rating(fullpath)
    df1d = pd.merge(dfmsd1,dfdsd1,on=['Contract Number'],how='left')
    df1d = pd.merge(df1d,dfsrd1,on=['Contract Number'],how='left')
    df1d = df1d.drop(columns=["Organization Type_x","Contract Name_x","Organization Marketing Name_x","Parent Organization_x","Organization Type_y","Contract Name_y","Organization Marketing Name_y","Parent Organization_y"])
    df1=df1.replace('//s*Yes//s*','Yes',regex=True)
    df1=df1.replace('//s*No//s*','No',regex=True)
    df1d=df1d.replace('//s*Yes//s*','Yes',regex=True)
    df1d=df1d.replace('//s*No//s*','No',regex=True)
    df1 = pd.merge(df1,df1d,on=["Contract Number","Organization Type","Contract Name","Organization Marketing Name","Parent Organization","SNP"],how='left')    
    df1 = df1.rename(columns={f"{year} Part C Summary Rating":"Part C Summary",f"{year} Overall Rating":"Overall",f"{year} Part D Summary Rating":"Part D Summary"})    
    df["Year"] = "2014 Fall"
    df1["Year"] = "2014 Spring"
    finaldf = pd.concat([df,df1])
    #spark = SparkSession.builder.appName("pandas to spark").getOrCreate()
    #sparkdf = spark.createDataFrame(finaldf)
    #sparkdf.show()
    for year in years:
        if (year <2020):
            df = get_early(year)
        else:
            df = get_late(year)
        #placeholderdf = spark.createDataFrame(df)
        #placeholderdf.show()
        newdf = pd.concat([finaldf,df],axis=0,join='outer')
        finaldf=newdf
    finaldf = finaldf.reset_index(drop=True)
    return finaldf

In [158]:
def clean_data(combodf1):
    for x in range(len(combodf1.keys())):
        combodf1[combodf1.keys()[x]] = combodf1[combodf1.keys()[x]].astype(str)
    combodf1 = combodf1.replace('\\s*Plan\\s*too\\s*new\\s*to\\s*be\\s*measured\\s*','Plan too new to be measured',regex=True)
    combodf1 = combodf1.replace('\\s*Plan\\s*too\\s*small\\s*to\\s*be\\s*measured\\s*','Plan too small to be measured',regex=True)
    combodf1 = combodf1.replace('\\s*Plan\\s*not\\s*required\\s*to\\s*report\\s*measure\\s*','Plan not required to report measure',regex=True)
    combodf1 = combodf1.replace('\\s*No\\s*data\\s*available\\s*','No data available',regex=True)
    combodf1 = combodf1.replace('Nodata available','No data available',regex=True)
    combodf1 = combodf1.replace('\\s*Not\\s*enough\\s*data\\s*available\\s*','Not enough data available',regex=True)
    combodf1 = combodf1.replace('\\s*Yes\\s*',True,regex=True)    
    combodf1 = combodf1.replace('\\s*No\\s*',False,regex=True)
    combodf1 = combodf1.replace('\\s*Benefit\\s*not\\s*offered\\s*by\\s*plan\\s*','Benefit not offered by plan',regex=True)        
    return combodf1

In [ ]:
def cull_data(df):
    

In [133]:
combodf = standardize_data()
combodf.to_csv("./Data/CombinedData.csv",header=True,index=False)

In [159]:
combodf = clean_data(combodf)
combodf.to_csv("./Data/CombinedDataCleaned.csv",header=True,index=False)

/var/folders/08/4n5vtwsd5jg4pw1917ng_prc0000gn/T/ipykernel_11010/2318447181.py:11: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  combodf1 = combodf1.replace('\\s*No\\s*',False,regex=True)


In [79]:
combodf.loc[combodf['C Breast Cancer Screening']=='Plan too small to be measured']

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions
8,H0141,Local CCP,"MCLAREN HEALTH PLAN, INC.",McLarenAdvantage,McLaren Health Care Corporation,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
18,H0294,Local CCP,CARE IMPROVEMENT PLUS WISCONSIN INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
31,H0408,Local CCP,UNITEDHEALTHCARE INSURANCE COMPANY,UnitedHealthcare,"UnitedHealth Group, Inc.",Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
36,H0490,Local CCP,"MOLINA HEALTHCARE OF OHIO, INC.",Molina Healthcare of Ohio,"Molina Healthcare, Inc.,",Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
37,H0502,1876 Cost,THE CONTRA COSTA HEALTH PLAN,Contra Costa Health Plan,Contra Costa Health Services,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12645,H9589,Local CCP,"CLEAR SPRING HEALTH (GA), INC.",Clear Spring Health,Group 1001,Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,Not enough data available,Not enough data available,Not enough data available,Not enough data available,Plan too small to be measured,Plan too small to be measured,Not enough data available,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured
12662,H9826,Local CCP,"COMMUNITY HEALTH CHOICE TEXAS, INC.",Community Health Choice,Harris County Hospital District,Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,2,2,1,2,Plan too small to be measured,Plan too small to be measured,4,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured
12663,H9827,Local CCP,"MOUNT CARMEL HEALTH PLAN OF NEW YORK, INC.",Trinity Health Plan New York,Trinity Health Corporation,Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,4,5,4,Not enough data available,Plan too small to be measured,Plan too small to be measured,5,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured
12667,H9888,Local CCP,DEVOTED HEALTH INSURANCE COMPANY OF ALABAMA INC,Devoted Health,"Devoted Health, Inc.",Plan too small to be measured,Plan too small to be measured,nan,nan,nan,...,1,2,2,Not enough data available,Plan too small to be measured,Plan too small to be measured,3,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured


In [160]:
combodf2 = combodf.copy()

rowswithnodata = []
datatosearch = ['1','2','3','4','5']
for i, r in combodf2.iterrows():
    row = r.to_list()
    if(any(val in row for val in datatosearch)):
        continue
    else:
        rowswithnodata.append(i)
rowswithnodata
#combodf2 = combodf2.replace('Not enough data available',0)


[0,
 4,
 7,
 9,
 10,
 14,
 17,
 28,
 34,
 35,
 57,
 58,
 60,
 65,
 86,
 95,
 108,
 116,
 119,
 122,
 123,
 127,
 130,
 134,
 136,
 138,
 162,
 180,
 181,
 182,
 184,
 196,
 197,
 200,
 205,
 206,
 216,
 217,
 218,
 219,
 222,
 231,
 234,
 260,
 265,
 266,
 270,
 287,
 308,
 310,
 317,
 318,
 327,
 332,
 342,
 350,
 367,
 369,
 375,
 385,
 391,
 403,
 407,
 408,
 427,
 451,
 469,
 473,
 505,
 510,
 511,
 512,
 514,
 518,
 522,
 523,
 525,
 526,
 527,
 531,
 533,
 536,
 539,
 540,
 544,
 545,
 548,
 550,
 555,
 559,
 561,
 566,
 567,
 568,
 569,
 574,
 575,
 576,
 577,
 579,
 581,
 585,
 586,
 588,
 589,
 590,
 591,
 592,
 594,
 595,
 596,
 597,
 598,
 600,
 601,
 603,
 605,
 607,
 610,
 614,
 615,
 617,
 618,
 630,
 631,
 632,
 633,
 634,
 635,
 638,
 641,
 644,
 648,
 649,
 650,
 651,
 652,
 654,
 656,
 659,
 661,
 662,
 665,
 677,
 681,
 684,
 686,
 687,
 691,
 694,
 705,
 711,
 712,
 734,
 735,
 737,
 742,
 763,
 772,
 785,
 793,
 796,
 799,
 800,
 804,
 807,
 811,
 813,
 815,
 839,


In [161]:
rowswithnodata.reverse()
for row in rowswithnodata:
    combodf2 = combodf2.drop(combodf2.index[row])
combodf2

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions
1,H0028,Local CCP,"CHA HMO, INC.","CHA HMO, Inc.",Humana Inc.,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2,2,4,4,2,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,3,4,5,4,2,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
5,H0108,Local CCP,"HUMANA HEALTH PLAN OF CALIFORNIA, INC.","Humana Health Plan of California, Inc.",Humana Inc.,3,3,5,4,2,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
6,H0117,Local CCP,"WELLCARE OF OHIO, INC.",WellCare,"WellCare Health Plans, Inc.",3,4,4,4,3,...,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12741,S6875,PDP,QCC INSURANCE COMPANY,Independence Blue Cross,"Independence Health Group, Inc.",Plan not required to report measure,Plan not required to report measure,nan,nan,nan,...,2,2,2,5,Plan not required to report measure,Plan not required to report measure,5,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
12742,S6946,PDP,CLEAR SPRING HEALTH INSURANCE COMPANY,Clear Spring Health,Group 1001,Plan not required to report measure,Plan not required to report measure,nan,nan,nan,...,1,1,1,2,Plan not required to report measure,Plan not required to report measure,2,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
12743,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,CAPITAL BLUE CROSS,Plan not required to report measure,Plan not required to report measure,nan,nan,nan,...,4,4,4,4,Plan not required to report measure,Plan not required to report measure,4,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
12744,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",Plan not required to report measure,Plan not required to report measure,nan,nan,nan,...,3,2,2,3,Plan not required to report measure,Plan not required to report measure,3,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure


In [162]:
combodf2 = combodf2.replace('nan','0')
combodf2

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions
1,H0028,Local CCP,"CHA HMO, INC.","CHA HMO, Inc.",Humana Inc.,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,Plan too new to be measured,...,0,0,0,0,0,0,0,0,0,0
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2,2,4,4,2,...,0,0,0,0,0,0,0,0,0,0
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,3,4,5,4,2,...,0,0,0,0,0,0,0,0,0,0
5,H0108,Local CCP,"HUMANA HEALTH PLAN OF CALIFORNIA, INC.","Humana Health Plan of California, Inc.",Humana Inc.,3,3,5,4,2,...,0,0,0,0,0,0,0,0,0,0
6,H0117,Local CCP,"WELLCARE OF OHIO, INC.",WellCare,"WellCare Health Plans, Inc.",3,4,4,4,3,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12741,S6875,PDP,QCC INSURANCE COMPANY,Independence Blue Cross,"Independence Health Group, Inc.",Plan not required to report measure,Plan not required to report measure,0,0,0,...,2,2,2,5,Plan not required to report measure,Plan not required to report measure,5,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
12742,S6946,PDP,CLEAR SPRING HEALTH INSURANCE COMPANY,Clear Spring Health,Group 1001,Plan not required to report measure,Plan not required to report measure,0,0,0,...,1,1,1,2,Plan not required to report measure,Plan not required to report measure,2,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
12743,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,CAPITAL BLUE CROSS,Plan not required to report measure,Plan not required to report measure,0,0,0,...,4,4,4,4,Plan not required to report measure,Plan not required to report measure,4,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
12744,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",Plan not required to report measure,Plan not required to report measure,0,0,0,...,3,2,2,3,Plan not required to report measure,Plan not required to report measure,3,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure


In [163]:
combodf2 = combodf2.replace('Plan too new to be measured',np.nan)
combodf2 = combodf2.dropna(how='any')
combodf2

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Diabetes Medications,D Medication Adherence for Hypertension (RAS antagonists),D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2,2,4,4,2,...,0,0,0,0,0,0,0,0,0,0
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,3,4,5,4,2,...,0,0,0,0,0,0,0,0,0,0
5,H0108,Local CCP,"HUMANA HEALTH PLAN OF CALIFORNIA, INC.","Humana Health Plan of California, Inc.",Humana Inc.,3,3,5,4,2,...,0,0,0,0,0,0,0,0,0,0
6,H0117,Local CCP,"WELLCARE OF OHIO, INC.",WellCare,"WellCare Health Plans, Inc.",3,4,4,4,3,...,0,0,0,0,0,0,0,0,0,0
8,H0141,Local CCP,"MCLAREN HEALTH PLAN, INC.",McLarenAdvantage,McLaren Health Care Corporation,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,Plan too small to be measured,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12741,S6875,PDP,QCC INSURANCE COMPANY,Independence Blue Cross,"Independence Health Group, Inc.",Plan not required to report measure,Plan not required to report measure,0,0,0,...,2,2,2,5,Plan not required to report measure,Plan not required to report measure,5,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
12742,S6946,PDP,CLEAR SPRING HEALTH INSURANCE COMPANY,Clear Spring Health,Group 1001,Plan not required to report measure,Plan not required to report measure,0,0,0,...,1,1,1,2,Plan not required to report measure,Plan not required to report measure,2,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
12743,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,CAPITAL BLUE CROSS,Plan not required to report measure,Plan not required to report measure,0,0,0,...,4,4,4,4,Plan not required to report measure,Plan not required to report measure,4,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure
12744,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",Plan not required to report measure,Plan not required to report measure,0,0,0,...,3,2,2,3,Plan not required to report measure,Plan not required to report measure,3,Plan not required to report measure,Plan not required to report measure,Plan not required to report measure


In [168]:
combodf2 = combodf2.replace('Plan too small to be measured','0')
combodf2 = combodf2.replace('Plan not required to report measure','0')
combodf2 = combodf2.replace('Plan not required to report ','0')
combodf2 = combodf2.replace('Nodata available','0')
combodf2 = combodf2.replace('Not enough data available','0')
combodf2 = combodf2.replace('Benefit not offered by plan','0')

In [154]:
combodf2.to_csv('./Data/CombinedDataSet0.csv',header=True,index=False)

In [28]:
badsnp = combodf2.loc[combodf['SNP']=='nan']
badsnp.to_csv("./Data/badSNP.csv",header=True,index=False)

In [165]:
cols = combodf2.keys().to_list()
for col in firstfive:
    cols.remove(col)
for col in lasttwo:
    cols.remove(col)
newkeys = firstfive+cols+lasttwo
newkeys

['Contract Number',
 'Organization Type',
 'Contract Name',
 'Organization Marketing Name',
 'Parent Organization',
 'C Breast Cancer Screening',
 'C Colorectal Cancer Screening',
 'C Cardiovascular Care – Cholesterol Screening',
 'C Diabetes Care – Cholesterol Screening',
 'C Glaucoma Testing',
 'C Annual Flu Vaccine',
 'C Improving or Maintaining Physical Health',
 'C Improving or Maintaining Mental Health',
 'C Monitoring Physical Activity',
 'C Adult BMI Assessment',
 'C Care for Older Adults – Medication Review',
 'C Care for Older Adults – Functional Status Assessment',
 'C Care for Older Adults – Pain Screening',
 'C Osteoporosis Management in Women who had a Fracture',
 'C Diabetes Care – Eye Exam',
 'C Diabetes Care – Kidney Disease Monitoring',
 'C Diabetes Care – Blood Sugar Controlled',
 'C Diabetes Care – Cholesterol Controlled',
 'C Controlling Blood Pressure',
 'C Rheumatoid Arthritis Management',
 'C Improving Bladder Control',
 'C Reducing the Risk of Falling',
 'C Pla

In [169]:
newdf = combodf2[newkeys]
newdf

,Contract Number,Organization Type,Contract Name,Organization Marketing Name,Parent Organization,C Breast Cancer Screening,C Colorectal Cancer Screening,C Cardiovascular Care – Cholesterol Screening,C Diabetes Care – Cholesterol Screening,C Glaucoma Testing,...,D Medication Adherence for Cholesterol (Statins),D MTM Program Completion Rate for CMR,C Medication Reconciliation Post-Discharge,C Statin Therapy for Patients with Cardiovascular Disease,D Statin Use in Persons with Diabetes (SUPD),C Controlling High Blood Pressure,C Transitions of Care,C Follow-up after Emergency Department Visit for People with Multiple High-Risk Chronic Conditions,Year,Overall
2,H0084,Local CCP,CARE IMPROVEMENT PLUS OF TEXAS INSURANCE COMPANY,Care Improvement Plus,"UnitedHealth Group, Inc.",2,2,4,4,2,...,0,0,0,0,0,0,0,0,2014 Fall,3
3,H0104,Local CCP,BLUE CROSS AND BLUE SHIELD OF ALABAMA,Blue Advantage (PPO),BlueCross BlueShield of Alabama,3,4,5,4,2,...,0,0,0,0,0,0,0,0,2014 Fall,3
5,H0108,Local CCP,"HUMANA HEALTH PLAN OF CALIFORNIA, INC.","Humana Health Plan of California, Inc.",Humana Inc.,3,3,5,4,2,...,0,0,0,0,0,0,0,0,2014 Fall,3.5
6,H0117,Local CCP,"WELLCARE OF OHIO, INC.",WellCare,"WellCare Health Plans, Inc.",3,4,4,4,3,...,0,0,0,0,0,0,0,0,2014 Fall,3
8,H0141,Local CCP,"MCLAREN HEALTH PLAN, INC.",McLarenAdvantage,McLaren Health Care Corporation,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2014 Fall,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12741,S6875,PDP,QCC INSURANCE COMPANY,Independence Blue Cross,"Independence Health Group, Inc.",0,0,0,0,0,...,2,5,0,0,5,0,0,0,2025,False
12742,S6946,PDP,CLEAR SPRING HEALTH INSURANCE COMPANY,Clear Spring Health,Group 1001,0,0,0,0,0,...,1,2,0,0,2,0,0,0,2025,False
12743,S8067,PDP,Avalon Insurance Company,Avalon Insurance Company,CAPITAL BLUE CROSS,0,0,0,0,0,...,4,4,0,0,4,0,0,0,2025,False
12744,S8841,PDP,"OPTUM INSURANCE OF OHIO, INC.","Optum Insurance of Ohio, Inc.","UnitedHealth Group, Inc.",0,0,0,0,0,...,2,3,0,0,3,0,0,0,2025,False


In [170]:
for col in cols:
    newdf[col] = newdf[col].astype(float)



In [171]:
newdf.to_csv("./Data/CombinedDataSet0ChangedToint")

In [20]:
len(combodf2)

12746

In [19]:
len(rowswithnodata)

2091